# ASSIGNMENT 3 

YASIR EMRE MARKAL   21069036


RECEP TAYYIP SEZER  20069006


MUHAMMET ENES SENOL 20069049

In [67]:
import pandas as pd
import numpy as np
import math
from pulp import *

# the seed number should be your student ID. 
# If it has letter, please ignore it and use only numbers in the ID
np.random.seed(21069036) 

# Requirment of the production (proudction hour)
RandomFtoD1 = np.round(np.random.uniform(low = 1, high = 10, size = 4),0).astype(int)
RandomFtoD2 = np.reshape(RandomFtoD1, (2,2))
BesiktasHour = pd.DataFrame(RandomFtoD2, columns=['T-shirts', 'Jeans'], 
                        index=["Cutting", "Sewing"])
BesiktasHour.columns=pd.MultiIndex.from_product([['Beşiktaş'],BesiktasHour.columns])

# Shipment cost from distribution center to store
RandomDtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 4),0).astype(int)
RandomDtoS2 = np.reshape(RandomDtoS1, (2,2))
DavutpasaHour = pd.DataFrame(RandomDtoS2, columns=['T-shirts', 'Jeans'], 
                        index=["Cutting", "Sewing"])
DavutpasaHour.columns=pd.MultiIndex.from_product([['Davutpaşa'],DavutpasaHour.columns])

# Your Required Information About Production Hour
df = pd.concat([BesiktasHour, DavutpasaHour], axis=1)
print(df)
df['Beşiktaş']['T-shirts']['Cutting']

        Beşiktaş       Davutpaşa      
        T-shirts Jeans  T-shirts Jeans
Cutting        8     2         9     4
Sewing         8     8         5     6


8

In [68]:


Clothes_ = ["BesiktasT-shirt", "BesiktasJeans","DavutpasaT-shirt","DavutpasaJeans"]

Skills_ = ["BesiktasCutting", "BesiktasSewing","DavutpasaCutting","DavutpasaSewing"]

Capacity_ = {"BesiktasCutting":80,
                 "BesiktasSewing":60,
                 "DavutpasaCutting":60,
                 "DavutpasaSewing":75,
                }

operation_times = {"BesiktasT-shirt": {"BesiktasCutting":df['Beşiktaş']['T-shirts']['Cutting'],
                                    "BesiktasSewing":df['Beşiktaş']['T-shirts']['Sewing'],
                                    },
                "BesiktasJeans": {"BesiktasCutting":df['Beşiktaş']['Jeans']['Cutting'],
                                  "BesiktasSewing":df['Beşiktaş']['Jeans']['Sewing'],
                                  
                                  },
                "DavutpasaT-shirt": {
                                    
                                    "DavutpasaCutting":df['Davutpaşa']['T-shirts']['Cutting'],
                                     "DavutpasaSewing":df['Davutpaşa']['T-shirts']['Sewing']
                                     },
                "DavutpasaJeans": {
                                   
                                    "DavutpasaCutting":df['Davutpaşa']['Jeans']['Cutting'],
                                    "DavutpasaSewing":df['Davutpaşa']['Jeans']['Sewing']}

                }

Profit = {"BesiktasT-shirt":10,"BesiktasJeans":15,"DavutpasaT-shirt":10,"DavutpasaJeans":15}

prob = LpProblem("Cloth making",LpMaximize)

x = LpVariable.dicts("x",Clothes_, lowBound=0, upBound=None, cat=LpInteger)

# Define the objective function
prob += lpSum(Profit[i]*x[i] for i in Clothes_)

# Define the constraints
for skills in Skills_[:2]:
    prob += lpSum(operation_times[cloths][skills]*x[cloths] for cloths in Clothes_[:2]) <= Capacity_[skills]
    
for skills in Skills_[2:]:
    prob += lpSum(operation_times[cloths][skills]*x[cloths] for cloths in Clothes_[2:]) <= Capacity_[skills]
    
prob += lpSum(4*x[i] for i in Clothes_) <= 120

# solve the problem
prob.solve()

C:\Users\yasir\anaconda3\lib\site-packages\pulp\pulp.py:1352: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


1

A) Production Quantities: 


In [69]:
# print the status of the solution
print("Status: ",LpStatus[prob.status])

# print the solution
for v in prob.variables():
        print(v.name, "=",v.varValue)


Status:  Optimal
x_BesiktasJeans = 7
x_BesiktasT_shirt = 0
x_DavutpasaJeans = 12
x_DavutpasaT_shirt = 0


B.1 Cutting times in Beşiktaş and in Davutpaşa

In [70]:
# print the cutting time for each Besiktas
print("Besiktas cutting time =", operation_times['BesiktasT-shirt']['BesiktasCutting']*value(x['BesiktasT-shirt']) + operation_times['BesiktasJeans']['BesiktasCutting']*value(x['BesiktasJeans']))

# print the cutting time for each Davutpasa
print("Davutpasa cutting time =", operation_times['DavutpasaT-shirt']['DavutpasaCutting']*value(x['DavutpasaT-shirt']) + operation_times['DavutpasaJeans']['DavutpasaCutting']*value(x['DavutpasaJeans']))


Besiktas cutting time = 14
Davutpasa cutting time = 48


B.2 Sewing times in Beşiktaş and in Davutpaşa 

In [71]:
# print the cutting time for each Besiktas
print("Besiktas sewing time =", operation_times['BesiktasT-shirt']['BesiktasSewing']*value(x['BesiktasT-shirt']) + operation_times['BesiktasJeans']['BesiktasSewing']*value(x['BesiktasJeans']))

# print the cutting time for each Davutpasa
print("Davutpasa sewing time =", operation_times['DavutpasaT-shirt']['DavutpasaSewing']*value(x['DavutpasaT-shirt']) + operation_times['DavutpasaJeans']['DavutpasaSewing']*value(x['DavutpasaJeans']))


Besiktas sewing time = 56
Davutpasa sewing time = 72


B.3 Amount of cloths in Beşiktaş and in Davutpaşa


In [72]:
besiktas_clothes = x["BesiktasT-shirt"].varValue + x["BesiktasJeans"].varValue
davutpasa_clothes = x["DavutpasaT-shirt"].varValue + x["DavutpasaJeans"].varValue

print("Total amount of clothes produced for Besiktas: ", besiktas_clothes)
print("Total amount of clothes produced for Davutpasa: ", davutpasa_clothes)


Total amount of clothes produced for Besiktas:  7
Total amount of clothes produced for Davutpasa:  12


C) Total profit. 

In [73]:
print("Profit = ",value(prob.objective))

Profit =  285
